In [35]:
from aocd import get_data

# Day 9: Disk Fragmenter

## Part One

In [1]:
example_input = "2333133121414131402"

In [2]:
import numpy as np

In [3]:
counts = np.array(list(example_input), dtype=int); counts

array([2, 3, 3, 3, 1, 3, 3, 1, 2, 1, 4, 1, 4, 1, 3, 1, 4, 0, 2])

In [4]:
len(counts)

19

In [5]:
values = np.array(list(''.join([f"{i}." for i in range(0, len(example_input)//2 + 1)]))); values

array(['0', '.', '1', '.', '2', '.', '3', '.', '4', '.', '5', '.', '6',
       '.', '7', '.', '8', '.', '9', '.'], dtype='<U1')

In [6]:
len(values)

20

In [23]:
blocks = np.repeat(values[:len(counts)], counts); blocks

array(['0', '0', '.', '.', '.', '1', '1', '1', '.', '.', '.', '2', '.',
       '.', '.', '3', '3', '3', '.', '4', '4', '.', '5', '5', '5', '5',
       '.', '6', '6', '6', '6', '.', '7', '7', '7', '.', '8', '8', '8',
       '8', '9', '9'], dtype='<U1')

In [24]:
''.join(blocks)

'00...111...2...333.44.5555.6666.777.888899'

In [25]:
def shift_blocks(arr):
    while '.' in arr:
        value = arr[-1]  # get last element 
        arr = np.delete(arr, -1)  # remove last element, numpy doesn't have pop()
        
        dot_index = np.where(arr == '.')[0][0]
        arr[dot_index] = value
    
    return arr
        

In [26]:
blocks = shift_blocks(blocks)

In [27]:
blocks

array(['0', '0', '9', '9', '8', '1', '1', '1', '8', '8', '8', '2', '7',
       '7', '7', '3', '3', '3', '6', '4', '4', '6', '5', '5', '5', '5',
       '6', '6'], dtype='<U1')

In [33]:
positions = np.arange(0, len(blocks))
checksum = np.sum(positions * blocks.astype(int))

In [34]:
checksum

np.int64(1928)

In [36]:
data = get_data(day=9, year=2024)

In [37]:
def calculate(data):
    counts = np.array(list(data), dtype=int)
    values = np.array(list(''.join([f"{i}." for i in range(0, len(data)//2 + 1)])))
    blocks = np.repeat(values[:len(counts)], counts)
    blocks = shift_blocks(blocks)
    positions = np.arange(0, len(blocks))
    checksum = np.sum(positions * blocks.astype(int))
    return checksum

In [38]:
calculate(data)

np.int64(11422652359)

In [40]:
counts = np.array(list(data), dtype=int)
values = np.array(list(''.join([f"{i}." for i in range(0, len(data)//2 + 1)])))
blocks = np.repeat(values[:len(counts)], counts)
blocks = shift_blocks(blocks)
positions = np.arange(0, len(blocks))
checksum = np.sum(positions * blocks.astype(int))

In [41]:
blocks

array(['0', '1', '1', ..., '3', '3', '8'], dtype='<U1')

In [42]:
len(blocks)

75910

In [43]:
positions

array([    0,     1,     2, ..., 75907, 75908, 75909])

In [44]:
blocks[:20]

array(['0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2',
       '2', '2', '2', '1', '2', '2', '2'], dtype='<U1')

In [45]:
data[:20]

'12346494626111444530'